In [1]:
#Import necessary packages
import math
import numpy as np
np.set_printoptions(suppress=True,
                    formatter={'all': lambda x: '%5.3f' % x})
import pandas as pd
from scipy.optimize import brute, fmin, minimize
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

#Install import_ipynb so we can read dependenies as jupyter notebooks
!pip install ipynb

#Import dependencies
from ipynb.fs.full.FFT_option_valuation import H93_call_value_FFT
from ipynb.fs.full.Vasicek_calibration import Vasicek_calibration , r_list
from ipynb.fs.full.Vasicek_zcb_valuation import zcb_price

In [2]:
#Import CSV file with option data as a panda dataframe
raw = pd.read_csv('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/option_data.csv')

#Convert variables Maturity and Date to pandas datetime objects
raw['Maturity']= pd.to_datetime(raw['Maturity'], format='%Y-%m-%d') 
raw['Date']= pd.to_datetime(raw['Date'], format='%Y-%m-%d')

In [3]:
#Create a copy so we do not have to reload the data set if we start over
data = raw.copy()
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Maturity  154 non-null    datetime64[ns]
 1   Strike    154 non-null    float64       
 2   Call      154 non-null    float64       
 3   Date      154 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2)
memory usage: 4.9 KB


,Maturity,Strike,Call,Date
0,2020-12-30,85.0,13.25,2020-12-28
1,2020-12-30,87.5,11.00,2020-12-28
2,2020-12-30,90.0,8.30,2020-12-28
3,2020-12-30,92.5,5.80,2020-12-28
4,2020-12-30,95.0,3.50,2020-12-28
...,...,...,...,...
149,2022-12-16,120.0,7.30,2020-12-28
150,2022-12-16,130.0,5.25,2020-12-28
151,2022-12-16,140.0,3.65,2020-12-28
152,2022-12-16,150.0,2.40,2020-12-28


In [4]:
#Create a variable, T, that shows how much, as a fraction of a year, that is left until maturity
data['diff'] = data['Maturity'] - data['Date']                     #number of days left to maturity
data['T']=(data['diff'].astype('timedelta64[D]').astype(int))/365  

#Drop unnecessary varibles
data.drop(['diff'], axis = 1, inplace=True)

#Drop max.rows option so we can see full dataframe to decide where implied vol gives NaN values
pd.set_option('display.max_rows', None)
data

,Maturity,Strike,Call,Date,T
0,2020-12-30,85.0,13.250,2020-12-28,0.005479
1,2020-12-30,87.5,11.000,2020-12-28,0.005479
2,2020-12-30,90.0,8.300,2020-12-28,0.005479
3,2020-12-30,92.5,5.800,2020-12-28,0.005479
4,2020-12-30,95.0,3.500,2020-12-28,0.005479
5,2020-12-30,97.5,1.350,2020-12-28,0.005479
6,2020-12-30,100.0,0.405,2020-12-28,0.005479
7,2021-01-08,85.0,13.500,2020-12-28,0.030137
8,2021-01-08,87.5,11.000,2020-12-28,0.030137
9,2021-01-08,90.0,8.500,2020-12-28,0.030137


In [5]:
#Reset maximum number of rows displayed to 10 rows
pd.set_option('display.max_rows', 10)

#Keep only options with less than 1 year to maturity
data = data[data['T'] <= 1]

#Keep only options with at least one week to maturity
data = data[data['T'] >= 7/365]
options = data
options

,Maturity,Strike,Call,Date,T
7,2021-01-08,85.0,13.500,2020-12-28,0.030137
8,2021-01-08,87.5,11.000,2020-12-28,0.030137
9,2021-01-08,90.0,8.500,2020-12-28,0.030137
10,2021-01-08,92.5,6.000,2020-12-28,0.030137
11,2021-01-08,95.0,3.900,2020-12-28,0.030137
...,...,...,...,...,...
132,2021-12-17,120.0,4.125,2020-12-28,0.969863
133,2021-12-17,130.0,2.425,2020-12-28,0.969863
134,2021-12-17,140.0,1.375,2020-12-28,0.969863
135,2021-12-17,150.0,0.825,2020-12-28,0.969863


In [6]:
#Initial short rate (Stibor 28.12.2020)
r0 = r_list[0]  

#Calibrate Short Rate Model to get calibrated estimates of CIR parameters used for zcb valuation
kappa_r, theta_r, sigma_r = Vasicek_calibration()

r = []
for row, option in options.iterrows():
    B0T = zcb_price([kappa_r, theta_r, sigma_r, r0, option['T']])
    r.append(-math.log(B0T) / option['T'])
options['r'] = r
options

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 153
         Function evaluations: 288


,Maturity,Strike,Call,Date,T,r
7,2021-01-08,85.0,13.500,2020-12-28,0.030137,-0.000491
8,2021-01-08,87.5,11.000,2020-12-28,0.030137,-0.000491
9,2021-01-08,90.0,8.500,2020-12-28,0.030137,-0.000491
10,2021-01-08,92.5,6.000,2020-12-28,0.030137,-0.000491
11,2021-01-08,95.0,3.900,2020-12-28,0.030137,-0.000491
...,...,...,...,...,...,...
132,2021-12-17,120.0,4.125,2020-12-28,0.969863,0.000721
133,2021-12-17,130.0,2.425,2020-12-28,0.969863,0.000721
134,2021-12-17,140.0,1.375,2020-12-28,0.969863,0.000721
135,2021-12-17,150.0,0.825,2020-12-28,0.969863,0.000721


In [7]:
##################################################################################################################
################################################ Preliminaries ###################################################

#Define Stock price 
S0 = 98.36

#Slice the dataframe
#Default option selects all options in the csv file

##################################################################################################################
######################################## Loss Function for the calibration #######################################

i = 0                                                              #Initialize iteration counter
min_MSE = 500                                                      #Initial min_MSE value
def H93_error_function_FFT(p0):
    ''' Error function for parameter calibration in BCC97 model via
    Lewis (2001) Fourier approach.

    Parameters
    ==========
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial, instantaneous variance

    Returns
    =======
    MSE: float
        mean squared error
    '''
    #kappa_v, theta_v, sigma_v, rho     , v0
    #a      , v_hat    eta      rho     , v
    global i, min_MSE                                           #Set global variables 
    kappa_v, theta_v, sigma_v, rho, v0 = p0                     #Define initial values of the parameter vector p0
    if kappa_v < 0.0 or theta_v < 0.005 or theta_v > 1.0 or sigma_v < 0.005 or sigma_v > 2.0 or rho < -1.0 or rho > 1.0:                       
        return 500.0
    if 2 * kappa_v * theta_v < sigma_v ** 2:      #Make sure that parameters adhere to their numerical boundaries
        return 500.0
    se = []
    delta = 2
    for row, option in options.iterrows():
        model_value = H93_call_value_FFT(S0, option['Strike'], option['T'],
                            option['r'], kappa_v, theta_v, sigma_v, rho, v0)
        se.append(delta**2 * (np.sqrt(1 + ((model_value - option['Call']) / delta)**2)) - 1)
    MSE = sum(se) / len(se)
    min_MSE = min(min_MSE, MSE)
    if i % 25 == 0:
        print('%4d |' % i, np.array(p0), '| %7.3f | %7.3f' % (MSE, min_MSE))
    i += 1
    return MSE

##################################################################################################################
################# Calibration of Heston (1993) parameters kappa_v, theta_v, sigma_v, rho, v0 #####################

def H93_calibration_full_FFT():
    ''' Calibrates H93 stochastic volatility model to market quotes. '''
    # first run with brute force
    # (scan sensible regions)
    #p0 = brute(H93_error_function,
    #            ((2.5, 10.6, 5.0),  # kappa_v
    #            (0.01, 0.041, 0.01),  # theta_v
    #            (0.05, 0.251, 0.1),  # sigma_v
    #            (-0.75, 0.01, 0.25),  # rho
    #            (0.01, 0.031, 0.01)),  # v0
    #            finish=None)
    
    p0 = [2, 0.3, 0.7, -0.3, 0.1]
    #kappa_v, theta_v, sigma_v, rho, v0

    # second run with local, convex minimization
    # (dig deeper where promising)
    opt = fmin(H93_error_function_FFT, p0,  
                 xtol=0.001, ftol=0.001, 
                 maxiter=1000, maxfun=1000)
    np.save('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/opt_sv', np.array(opt))
    return opt

##################################################################################################################
################### Calculation of Heston (1993) option values with stochastic interest rates  ######################

def H93_calculate_model_values_FFT(p0):
    ''' Calculates all model values given parameter vector p0. '''
    kappa_v, theta_v, sigma_v, rho, v0 = p0  
    values = []
    for row, option in options.iterrows():
        model_value = H93_call_value_FFT(S0, option['Strike'], option['T'],
                            option['r'], kappa_v, theta_v, sigma_v, rho, v0)
        values.append(model_value)
    return np.array(values) 

##################################################################################################################
################################################# Run calibration ################################################

if __name__ == '__main__': 
    
    #Run and print the calibration
    opt = H93_calibration_full_FFT()
    print('')
    print("Calibrated parameter vector: ", opt)

    #Create pandas column for Heston (1993) call options values based on calibrated data
    options['Model'] = H93_calculate_model_values_FFT(opt)

   0 | [2.000 0.300 0.700 -0.300 0.100] |   4.090 |   4.090
  25 | [2.171 0.190 0.823 -0.343 0.100] |   3.363 |   3.201
  50 | [2.706 0.089 0.664 -0.427 0.095] |   3.036 |   3.035
  75 | [2.773 0.086 0.629 -0.429 0.097] |   3.035 |   3.032
 100 | [3.156 0.080 0.420 -0.460 0.101] |   3.028 |   3.028
 125 | [3.415 0.083 0.303 -0.485 0.096] |   3.025 |   3.024
 150 | [3.460 0.087 0.296 -0.488 0.092] |   3.024 |   3.024
 175 | [3.468 0.087 0.297 -0.488 0.091] |   3.024 |   3.024
 200 | [3.470 0.087 0.297 -0.487 0.091] |   3.024 |   3.024
 225 | [3.471 0.087 0.300 -0.486 0.092] |   3.024 |   3.024
 250 | [3.560 0.087 0.320 -0.466 0.092] |   3.024 |   3.024
 275 | [3.766 0.087 0.360 -0.424 0.092] |   3.024 |   3.024
 300 | [3.766 0.087 0.358 -0.425 0.092] |   3.024 |   3.024
 325 | [3.773 0.087 0.359 -0.423 0.092] |   3.024 |   3.024
 350 | [3.777 0.087 0.358 -0.424 0.092] |   3.024 |   3.024
 375 | [3.812 0.087 0.354 -0.428 0.092] |   3.024 |   3.024
 400 | [3.885 0.087 0.360 -0.423 0.092] 

In [8]:
#Drop max.rows option so we can see full dataframe 
pd.set_option('display.max_rows', None)
options

,Maturity,Strike,Call,Date,T,r,Model
7,2021-01-08,85.0,13.500,2020-12-28,0.030137,-0.000491,13.366910
8,2021-01-08,87.5,11.000,2020-12-28,0.030137,-0.000491,10.892409
9,2021-01-08,90.0,8.500,2020-12-28,0.030137,-0.000491,8.474010
10,2021-01-08,92.5,6.000,2020-12-28,0.030137,-0.000491,6.188730
11,2021-01-08,95.0,3.900,2020-12-28,0.030137,-0.000491,4.156801
12,2021-01-08,97.5,2.150,2020-12-28,0.030137,-0.000491,2.510738
13,2021-01-08,100.0,0.900,2020-12-28,0.030137,-0.000491,1.333789
14,2021-01-08,102.5,0.405,2020-12-28,0.030137,-0.000491,0.610960
15,2021-01-08,105.0,0.405,2020-12-28,0.030137,-0.000491,0.237506
16,2021-01-15,82.5,16.000,2020-12-28,0.049315,-0.000437,15.876101
